In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
import subprocess
import os
# python version: 3.11.9

In [2]:
# presets, functions

# preset values for the analysis:
#____________________________________________________________________________________________________________________
# physical constants
Z = 26
A = 56
mass_nucleon = 0.938273 
# mass_nucleus = 11.178
mass_nucleus = A * 0.931494
alpha_fine = 1 / 137

# three-momentum bin centers
qvcenters = [0.100, 0.148, 0.167, 0.205, 0.240, 0.300, 0.380, 0.475, 0.570, 0.649, 0.756, 0.991, 1.619, 1.921, 2.213, 2.500, 2.783, 3.500]
# three-momentum edges
qvbins = [0.063, 0.124, 0.158, 0.186, 0.223, 0.270, 0.340, 0.428, 0.523, 0.609, 0.702, 0.878, 1.302, 1.770, 2.067, 2.357, 2.642, 2.923, 4.500]
# four-momentum squared bin names, in string format
qvbin_names = ['[0.063,0.124]', '[0.124,0.158]', '[0.158,0.186]', '[0.186,0.223]', '[0.223,0.270]', '[0.270,0.340]', '[0.340,0.428]', '[0.428,0.523]', '[0.523,0.609]',
                '[0.609,0.702]', '[0.702,0.878]', '[0.878,1.302]', '[1.302,1.770]', '[1.770,2.067]', '[2.067,2.357]', '[2.357,2.642]', '[2.642,2.923]', '[2.923,4.500]']

# four-momentum squared bin centers
Q2centers = [0.010, 0.020, 0.026, 0.040, 0.056, 0.093, 0.120, 0.160, 0.265, 0.380, 0.500, 0.800, 1.250, 1.750, 2.250, 2.750, 3.250, 3.750]
# four-momentum squared bin edges
Q2bins = [0.004, 0.015, 0.025, 0.035, 0.045, 0.070, 0.100, 0.145, 0.206, 0.322, 0.438, 0.650, 1.050, 1.500, 2.000, 2.500, 3.000, 3.500, 4.000]
# four-momentum squared bin names, in string format
Q2bin_names = ['[0.004,0.015]', '[0.015,0.025]', '[0.025,0.035]', '[0.035,0.045]', '[0.045,0.070]', '[0.070,0.100]', '[0.100,0.145]', '[0.145,0.206]', '[0.206,0.322]',
                '[0.322,0.438]', '[0.438,0.650]', '[0.650,1.050]', '[1.050,1.500]', '[1.500,2.000]', '[2.000,2.500]', '[2.500,3.000]', '[3.000,3.500]', '[3.500,4.000]']

def round_sig_3(x):
    return '%s' % float('%.3g' % x)

def append_row(df, row):
    return pd.concat([df, pd.DataFrame([row], columns=row.index)]).reset_index(drop=True)

def linear_model(x, a, b):
    return a * x + b

# read dataframe from csv file
def prepare_df(filepath):
    # read the csv file:
    df = pd.read_csv(filepath)

    df["Veff"]=0.0089
    
    df["Eeff"] = df["E0"] + df["Veff"]
    df["Ep"] = df["E0"] - df["nu"]
    df["Ep_eff"] = df["Ep"] + df["Veff"]

    # calculate normalized cross section:
    df['normalization'] = 1.0
    df['normError'] = 0.0
    df_norm = pd.read_csv('Data/Fe56_Norm.csv')
    for _, row in df_norm.iterrows():
        mask = pd.Series([True] * len(df))
        if pd.notna(row['E0']):
            mask &= df['E0'] == row['E0']
        if pd.notna(row['ThetaDeg']):
            mask &= df['ThetaDeg'] == row['ThetaDeg']
        if pd.notna(row['dataSet']):
            mask &= df['dataSet'] == row['dataSet']
        df.loc[mask, 'normalization'] = row['normalization']
        try:
            df.loc[mask, 'system_err'] = float(row['system_err'])
        except (ValueError, TypeError):
            df.loc[mask, 'system_err'] = np.nan
    df.loc[df['dataSet'] == 'Meziani:1984is', 'system_err'] = np.where(df.loc[df['dataSet'] == 'Meziani:1984is', 'Ep_eff'] > 0.2, 0.037, 0.037 + 0.016 * (0.2 - df.loc[df['dataSet'] == 'Meziani:1984is', 'Ep_eff']) / 0.15)
    df.loc[df['dataSet'] == 'Altemus:1973', 'system_err'] = np.where(df.loc[df['dataSet'] == 'Altemus:1973', 'Ep_eff'] > 0.2, 0.037, 0.037 + 0.016 * (0.2 - df.loc[df['dataSet'] == 'Altemus:1973', 'Ep_eff']) / 0.15)
    df['normCross'] = df['cross'] * df['normalization']
    df['normCrossError'] = (np.sqrt(df['error']**2 + ((df['system_err'] * df['cross'])**2))) * df['normalization']
    print(df.loc[df['normalization'] == 1, 'dataSet'].unique())
    
    # calculate the kinematic variables:
    df["ThetaRad"] = df["ThetaDeg"]*np.pi/180
    df["sin2(T/2)"] = (np.sin(df["ThetaRad"] / 2))**2
    df["cos2(T/2)"] = (np.cos(df["ThetaRad"] / 2))**2
    df["tan2(T/2)"] = (np.tan(df["ThetaRad"] / 2))**2

    df["nuel"] = df["E0"] - df["E0"] / (1 + 2 * df["E0"] * df["sin2(T/2)"] / mass_nucleus)
    df["Ex"] = df["nu"] - (df["E0"] - df["E0"] / (1 + 2 * df["E0"] * df["sin2(T/2)"] / mass_nucleus))

    df["R"] = 1.1 * (df["A"])**(1/3) + 0.86 / ((df["A"])**(1/3))

    df["F2foc"] = (df["Eeff"] / df["E0"])**2
    df["Q2"] = 4 * df["E0"] * (df["Ep"]) * df["sin2(T/2)"]
    df["Q2eff"] = 4 * df["Eeff"] * df["Ep_eff"] * df["sin2(T/2)"]
    df["qv2"] = df["nu"]**2 + df["Q2eff"]
    df["qv"] = np.sqrt(df["qv2"])

    df["W2"] = mass_nucleon**2 + 2 * mass_nucleon * df["nu"] - df["Q2eff"]
    df["epsilon"] = 1 / (1 + 2 * (1 + (df["nu"]**2) / df["Q2eff"]) * df["tan2(T/2)"])

    df["gamma"] = alpha_fine * df["Ep_eff"] * (df["W2"] - mass_nucleon**2) / (( 4 * ((np.pi)**2) * df["Q2eff"] * mass_nucleon * df["E0"]) * (1 - df["epsilon"]))
    df["Sig_R"] = df["normCross"] / df["gamma"]
    df["D_sig_R"] = df["error"] / df["gamma"]
    df["Sig_mott"] = 4 * (alpha_fine**2) * (df["Ep"]**2) * df["cos2(T/2)"] / (df["Q2"]**2)
    df["Sig_mott_eff"] = df["Sig_mott"] * df["E0"] / df["Eeff"]

    # Calculate the Rosenbluth quantity:
    df["H"] = (df["qv2"]**2) / (4 * (alpha_fine**2) * (df["Ep_eff"]**2) * (df["cos2(T/2)"] + 2 * (df["qv2"] / df["Q2eff"]) * df["sin2(T/2)"]))
    df["Hcc"] = df["H"] / df["F2foc"]
    df["Hstar_Sig(nb)"] = df["H"] * df["normCross"]
    df["Hstar_error(nb)"] = df["H"] * df["normCrossError"]
    df["Hstar_Sig(GeV)"] = df["Hstar_Sig(nb)"] / ((0.1973269**2) * 10000000)
    df["Hstar_error(GeV)"] = df["Hstar_error(nb)"] / ((0.1973269**2) * 10000000)
    df["Hcc_Sig(nb)"] = df["Hcc"] * df["normCross"]
    df["Hcc_error(nb)"] = df["Hcc"] * df["normCrossError"]
    df["Hcc_Sig(GeV)"] = df["Hcc_Sig(nb)"] / ((0.1973269**2) * 10000000)
    df["Hcc_error(GeV)"] = df["Hcc_error(nb)"] / ((0.1973269**2) * 10000000)
    # we will fit "Hcc_Sig(GeV)" vs "epsilon" to the linear model to get the Rosenbluth slope and intercept

    # subdivide the data into bins
    df['qvbin'] = 0
    df['qvcenter'] = 0
    df['Q2bin'] = 0
    df['Q2center'] = 0
    df["qvbin"] = pd.cut(x=df["qv"], bins=qvbins, labels=qvbin_names, right=True)
    df["qvcenter"] = pd.cut(x=df["qv"], bins=qvbins, labels=qvcenters, right=True)
    df['qvcenter'] = pd.to_numeric(df['qvcenter'])
    df["Q2bin"] = pd.cut(x=df["Q2eff"], bins=Q2bins, labels=Q2bin_names, right=True)
    df["Q2center"] = pd.cut(x=df["Q2eff"], bins=Q2bins, labels=Q2centers, right=True)
    df['Q2center'] = pd.to_numeric(df['Q2center'])
    df = df.dropna()
    # now every row of data has a bin (Q2/qv) label

    # this is for bc_qv_ex and bc_q2_ex
    df['ExA'] = np.where(df['Ex'] >= 0.04, df['Ex'], 0.04)
    df['nuA'] = np.where(df['Ex'] >= 0.04, df['nu'], df['nu'] + 0.04 - df['Ex'])
    df["Q2effA"] = 4 * df["Eeff"] * (df["Eeff"] - df['nuA']) * df["sin2(T/2)"]
    df["qv2A"] = df["nuA"]**2 + df["Q2effA"]
    df["epsilonA"] = 1 / (1 + 2 * (1 + (df["nuA"]**2) / df["Q2effA"]) * df["tan2(T/2)"])


    return df

In [3]:
# read csv file
df = prepare_df('Data/Fe56.csv')
df = df[~((df['dataSet'] == "Chen:1990kq") & (df['E0'] == 2.7) & (df['W2'] < 1))]
df = df[~((df['dataSet'] == "Chen:1990kq") & (df['E0'] == 3.3) & (df['W2'] < 1))]
df = df[~((df['dataSet'] == "Chen:1990kq") & (df['E0'] == 3.6) & (df['W2'] < 1))]
df = df[~((df['dataSet'] == "Chen:1990kq") & (df['E0'] == 3.9) & (df['W2'] < 1))]
df = df[~((df['dataSet'] == "Chen:1990kq") & (df['E0'] == 4.3) & (df['W2'] < 0.8))]
df = df[~((df['dataSet'] == "Chen:1990kq") & (df['E0'] == 0.9) & (df['W2'] > 1))]
df = df[~((df['dataSet'] == "Chen:1990kq") & (df['E0'] == 1.1) & (df['W2'] > 1))]
df = df[~((df['dataSet'] == "Sealock:1989nx") & (df['E0'] == 1.299) & (df['W2'] < 0.75))]
df

[]


,Z,A,E0,ThetaDeg,nu,cross,error,dataSet,Veff,Eeff,...,Hcc_error(GeV),qvbin,qvcenter,Q2bin,Q2center,ExA,nuA,Q2effA,qv2A,epsilonA
0,26,56,0.1503,90.00,0.00123,85208.0,1821.5,Altemus:1973,0.0089,0.1592,...,2.943884,"[0.223,0.270]",0.240,"[0.045,0.070]",0.056,0.040000,0.040432,0.037816,0.039451,0.323996
1,26,56,0.1503,90.00,0.00362,119440.0,1139.8,Altemus:1973,0.0089,0.1592,...,3.787135,"[0.186,0.223]",0.205,"[0.045,0.070]",0.056,0.040000,0.040432,0.037816,0.039451,0.323996
2,26,56,0.1503,90.00,0.00742,95248.0,1775.1,Altemus:1973,0.0089,0.1592,...,3.254174,"[0.186,0.223]",0.205,"[0.045,0.070]",0.056,0.040000,0.040432,0.037816,0.039451,0.323996
3,26,56,0.1503,90.00,0.00961,88073.0,1193.1,Altemus:1973,0.0089,0.1592,...,2.908262,"[0.186,0.223]",0.205,"[0.045,0.070]",0.056,0.040000,0.040432,0.037816,0.039451,0.323996
4,26,56,0.1503,90.00,0.01869,95613.0,1465.7,Altemus:1973,0.0089,0.1592,...,3.288240,"[0.186,0.223]",0.205,"[0.035,0.045]",0.040,0.040000,0.040432,0.037816,0.039451,0.323996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4824,26,56,5.1500,22.98,2.89470,761.0,11.6,E04001,0.0089,5.1589,...,4.537154,"[2.923,4.500]",3.500,"[1.500,2.000]",1.750,2.854664,2.894700,1.853946,10.233234,0.686749
4825,26,56,5.1500,22.98,2.90950,755.0,11.7,E04001,0.0089,5.1589,...,4.622677,"[2.923,4.500]",3.500,"[1.500,2.000]",1.750,2.869464,2.909500,1.841827,10.307018,0.683785
4826,26,56,5.1500,22.98,2.92440,767.0,12.5,E04001,0.0089,5.1589,...,4.860375,"[2.923,4.500]",3.500,"[1.500,2.000]",1.750,2.884364,2.924400,1.829627,10.381743,0.680778
4827,26,56,5.1500,22.98,2.93920,777.0,13.1,E04001,0.0089,5.1589,...,5.079429,"[2.923,4.500]",3.500,"[1.500,2.000]",1.750,2.899164,2.939200,1.817509,10.456406,0.677769


In [4]:
response_columns = ['i','RTTOT','RLTOT','RTQE','RLQE','RTNS','RLNS']
C12_factor = Z / 6
# 
QENS_shift = 0.013
NS_factor = 0.5
df = df[df['nu'] > QENS_shift].reset_index(drop=True)
df_shift = df.copy()
df_shift['nu'] = df['nu'] - QENS_shift
#

# calculate bc_qv_nu
df[['qvcenter','nu']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_nu.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['qvcenter','nu']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_qv_nu.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE'] - df_response['RTNS'] + df_response_shift['RTNS'] * NS_factor
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] - df_response['RLNS'] + df_response_shift['RLNS'] * NS_factor
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE']
df_response['RTNS'] = df_response_shift['RTNS'] * NS_factor
df_response['RLNS'] = df_response_shift['RLNS'] * NS_factor
#
df['RL_qvc_nu'] = df.index.map(df_response.set_index('i')['RLTOT']) # qvc: qv at bin center
df['RT_qvc_nu'] = df.index.map(df_response.set_index('i')['RTTOT']) # qvd: qv of data

df[['qv','nu']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_nu.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['qv','nu']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_qv_nu.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE'] - df_response['RTNS'] + df_response_shift['RTNS'] * NS_factor
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] - df_response['RLNS'] + df_response_shift['RLNS'] * NS_factor
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE']
df_response['RTNS'] = df_response_shift['RTNS'] * NS_factor
df_response['RLNS'] = df_response_shift['RLNS'] * NS_factor
#
df['RL_qvd_nu'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_qvd_nu'] = df.index.map(df_response.set_index('i')['RTTOT'])
print('RL RT qv_nu done.')

# calculate bc_qv_ex
df[['qvcenter','ExA']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_ex.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['qvcenter','ExA']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_qv_ex.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE'] - df_response['RTNS'] + df_response_shift['RTNS'] * NS_factor
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] - df_response['RLNS'] + df_response_shift['RLNS'] * NS_factor
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE']
df_response['RTNS'] = df_response_shift['RTNS'] * NS_factor
df_response['RLNS'] = df_response_shift['RLNS'] * NS_factor
#
df['RL_qvc_ex'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_qvc_ex'] = df.index.map(df_response.set_index('i')['RTTOT'])

df[['qv','ExA']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_ex.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['qv','ExA']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_qv_ex.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE'] - df_response['RTNS'] + df_response_shift['RTNS'] * NS_factor
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] - df_response['RLNS'] + df_response_shift['RLNS'] * NS_factor
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE']
df_response['RTNS'] = df_response_shift['RTNS'] * NS_factor
df_response['RLNS'] = df_response_shift['RLNS'] * NS_factor
#
df['RL_qvd_ex'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_qvd_ex'] = df.index.map(df_response.set_index('i')['RTTOT'])
print('RL RT qv_ex done.')

# calculate bc_qv_w2
df[['qvcenter','W2']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_w2.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['qvcenter','W2']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_qv_w2.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE'] - df_response['RTNS'] + df_response_shift['RTNS'] * NS_factor
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] - df_response['RLNS'] + df_response_shift['RLNS'] * NS_factor
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE']
df_response['RTNS'] = df_response_shift['RTNS'] * NS_factor
df_response['RLNS'] = df_response_shift['RLNS'] * NS_factor
#
df['RL_qvc_w2'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_qvc_w2'] = df.index.map(df_response.set_index('i')['RTTOT'])

df[['qv','W2']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_qv_w2.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['qv','W2']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_qv_w2.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE'] - df_response['RTNS'] + df_response_shift['RTNS'] * NS_factor
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] - df_response['RLNS'] + df_response_shift['RLNS'] * NS_factor
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE']
df_response['RTNS'] = df_response_shift['RTNS'] * NS_factor
df_response['RLNS'] = df_response_shift['RLNS'] * NS_factor
#
df['RL_qvd_w2'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_qvd_w2'] = df.index.map(df_response.set_index('i')['RTTOT'])
print('RL RT qv_w2 done.')

# calculate bc_q2_nu
df[['Q2center','nu']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_nu.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['Q2center','nu']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_q2_nu.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE'] - df_response['RTNS'] + df_response_shift['RTNS'] * NS_factor
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] - df_response['RLNS'] + df_response_shift['RLNS'] * NS_factor
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE']
df_response['RTNS'] = df_response_shift['RTNS'] * NS_factor
df_response['RLNS'] = df_response_shift['RLNS'] * NS_factor
#
df['RL_q2c_nu'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2c_nu'] = df.index.map(df_response.set_index('i')['RTTOT'])

df[['Q2eff','nu']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_nu.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['Q2eff','nu']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_q2_nu.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE'] - df_response['RTNS'] + df_response_shift['RTNS'] * NS_factor
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] - df_response['RLNS'] + df_response_shift['RLNS'] * NS_factor
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE']
df_response['RTNS'] = df_response_shift['RTNS'] * NS_factor
df_response['RLNS'] = df_response_shift['RLNS'] * NS_factor
#
df['RL_q2d_nu'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2d_nu'] = df.index.map(df_response.set_index('i')['RTTOT'])
print('RL RT q2_nu done.')

# calculate bc_q2_ex
df[['Q2center','ExA']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_ex.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['Q2center','ExA']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_q2_ex.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE'] - df_response['RTNS'] + df_response_shift['RTNS'] * NS_factor
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] - df_response['RLNS'] + df_response_shift['RLNS'] * NS_factor
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE']
df_response['RTNS'] = df_response_shift['RTNS'] * NS_factor
df_response['RLNS'] = df_response_shift['RLNS'] * NS_factor
#
df['RL_q2c_ex'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2c_ex'] = df.index.map(df_response.set_index('i')['RTTOT'])

df[['Q2eff','ExA']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_ex.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['Q2eff','ExA']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_q2_ex.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE'] - df_response['RTNS'] + df_response_shift['RTNS'] * NS_factor
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] - df_response['RLNS'] + df_response_shift['RLNS'] * NS_factor
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE']
df_response['RTNS'] = df_response_shift['RTNS'] * NS_factor
df_response['RLNS'] = df_response_shift['RLNS'] * NS_factor
#
df['RL_q2d_ex'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2d_ex'] = df.index.map(df_response.set_index('i')['RTTOT'])
print('RL RT q2_ex done.')

# calculate bc_q2_w2
df[['Q2center','W2']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_w2.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['Q2center','W2']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_q2_w2.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE'] - df_response['RTNS'] + df_response_shift['RTNS'] * NS_factor
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] - df_response['RLNS'] + df_response_shift['RLNS'] * NS_factor
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE']
df_response['RTNS'] = df_response_shift['RTNS'] * NS_factor
df_response['RLNS'] = df_response_shift['RLNS'] * NS_factor
#
df['RL_q2c_w2'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2c_w2'] = df.index.map(df_response.set_index('i')['RTTOT'])

df[['Q2eff','W2']].to_csv('input.txt',index=True,header=False,sep=' ')
with open('output.txt', 'w') as output_file:
    subprocess.run(['./response_q2_w2.exe', 'input.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response = pd.read_csv('output.txt', sep='\s+', header=None, names=response_columns)
#
df_shift[['Q2eff','W2']].to_csv('input_shift.txt',index=True,header=False,sep=' ')
with open('output_shift.txt', 'w') as output_file:
    subprocess.run(['./response_q2_w2.exe', 'input_shift.txt'], stdout=output_file) 
subprocess.run(['sleep', '0.5'])
df_response_shift = pd.read_csv('output_shift.txt', sep='\s+', header=None, names=response_columns)
df_response['RTTOT'] = df_response['RTTOT'] - df_response['RTQE'] + df_response_shift['RTQE'] - df_response['RTNS'] + df_response_shift['RTNS'] * NS_factor
df_response['RLTOT'] = df_response['RLTOT'] - df_response['RLQE'] + df_response_shift['RLQE'] - df_response['RLNS'] + df_response_shift['RLNS'] * NS_factor
df_response['RTQE'] = df_response_shift['RTQE']
df_response['RLQE'] = df_response_shift['RLQE']
df_response['RTNS'] = df_response_shift['RTNS'] * NS_factor
df_response['RLNS'] = df_response_shift['RLNS'] * NS_factor
#
df['RL_q2d_w2'] = df.index.map(df_response.set_index('i')['RLTOT'])
df['RT_q2d_w2'] = df.index.map(df_response.set_index('i')['RTTOT'])
print('RL RT q2_w2 done.')


<>:16: SyntaxWarning: invalid escape sequence '\s'
<>:22: SyntaxWarning: invalid escape sequence '\s'
<>:37: SyntaxWarning: invalid escape sequence '\s'
<>:43: SyntaxWarning: invalid escape sequence '\s'
<>:60: SyntaxWarning: invalid escape sequence '\s'
<>:66: SyntaxWarning: invalid escape sequence '\s'
<>:81: SyntaxWarning: invalid escape sequence '\s'
<>:87: SyntaxWarning: invalid escape sequence '\s'
<>:104: SyntaxWarning: invalid escape sequence '\s'
<>:110: SyntaxWarning: invalid escape sequence '\s'
<>:125: SyntaxWarning: invalid escape sequence '\s'
<>:131: SyntaxWarning: invalid escape sequence '\s'
<>:148: SyntaxWarning: invalid escape sequence '\s'
<>:154: SyntaxWarning: invalid escape sequence '\s'
<>:169: SyntaxWarning: invalid escape sequence '\s'
<>:175: SyntaxWarning: invalid escape sequence '\s'
<>:192: SyntaxWarning: invalid escape sequence '\s'
<>:198: SyntaxWarning: invalid escape sequence '\s'
<>:213: SyntaxWarning: invalid escape sequence '\s'
<>:219: SyntaxWarnin

RL RT qv_nu done.
RL RT qv_ex done.
RL RT qv_w2 done.
RL RT q2_nu done.
RL RT q2_ex done.
RL RT q2_w2 done.


In [5]:
# determine bin-centering corrections
df["bc_qv_nu"] = 1.0
df["bc_qv_ex"] = 1.0
df["bc_qv_w2"] = 1.0
df["bc_q2_nu"] = 1.0
df["bc_q2_ex"] = 1.0
df["bc_q2_w2"] = 1.0
for qvcenter in qvcenters:
    picked = df.loc[df["qvcenter"] == qvcenter]
    for index, row in picked.iterrows():
        if row['qvcenter'] != float('NaN') and row['Ex'] >= 0.025:
            try:
                df.loc[index, 'bc_qv_nu'] = (row['epsilon'] * row['RL_qvc_nu'] + 0.5 * ((qvcenter**2) / (qvcenter**2 - row['nu']**2)) * row['RT_qvc_nu']) / (row['epsilon'] * row['RL_qvd_nu'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_qvd_nu'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_qv_nu due to zero denominator.')
            try:
                df.loc[index, 'bc_qv_ex'] = (row['epsilonA'] * row['RL_qvc_ex'] + 0.5 * ((qvcenter**2) / (qvcenter**2 - row['nuA']**2)) * row['RT_qvc_ex']) / (row['epsilonA'] * row['RL_qvd_ex'] + 0.5 * (row['qv2A'] / row['Q2effA']) * row['RT_qvd_ex'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_qv_ex due to zero denominator.')
            try:
                df.loc[index, 'bc_qv_w2'] = (row['epsilon'] * row['RL_qvc_w2'] + 0.5 * ((qvcenter**2) / (2 * mass_nucleon * np.sqrt(qvcenter**2 + row['W2']) - row['W2'] - mass_nucleon**2)) * row['RT_qvc_w2']) / (row['epsilon'] * row['RL_qvd_w2'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_qvd_w2'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_qv_w2 due to zero denominator.')

for Q2center in Q2centers:
    picked = df.loc[df["Q2center"] == Q2center]
    for index, row in picked.iterrows():
        if row['Q2center'] != float('NaN') and row['Ex'] >= 0.025:
            nu = (row['W2'] + Q2center - mass_nucleon**2) / (2 * mass_nucleon)
            qv2center = Q2center + nu**2
            try:
                df.loc[index, 'bc_q2_nu'] = (row['epsilon'] * row['RL_q2c_nu'] + 0.5 * ((Q2center + row["nu"]**2) / (Q2center)) * row['RT_q2c_nu']) / (row['epsilon'] * row['RL_q2d_nu'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_q2d_nu'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_q2_nu due to zero denominator.')
            try:
                df.loc[index, 'bc_q2_ex'] = (row['epsilonA'] * row['RL_q2c_ex'] + 0.5 * ((Q2center + row["nuA"]**2) / (Q2center)) * row['RT_q2c_ex']) / (row['epsilonA'] * row['RL_q2d_ex'] + 0.5 * (row['qv2A'] / row['Q2effA']) * row['RT_q2d_ex'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_q2_ex due to zero denominator.')
            try:
                df.loc[index, 'bc_q2_w2'] = (row['epsilon'] * row['RL_q2c_w2'] + 0.5 * (qv2center / (Q2center)) * row['RT_q2c_w2']) / (row['epsilon'] * row['RL_q2d_w2'] + 0.5 * (row['qv2'] / row['Q2eff']) * row['RT_q2d_w2'])
            except ZeroDivisionError:
                print(f'Skipping index {index} for bc_q2_w2 due to zero denominator.')

In [6]:
# save dataframe to csv
df.to_csv('Data/df_Fe56.csv',index=False)